This notebook tests if bx-python interval intersection results match bedtools "intersect" and "window" results using the bedtools test bed files. We use bx-python to find which intervals overlap only and that's the only functionality tested here.

In [65]:
!cat "a.bed"

chr1	10	20	a1	1	+
chr1	100	200	a2	2	-


In [66]:
!cat "b.bed"

chr1	20	30	b1	1	+
chr1	90	101	b2	2	-
chr1	100	110	b3	3	+
chr1	200	210	b4	4	+


In [67]:
from bx.intervals.intersection import Interval, IntervalTree
import collections
import numpy as np

def intersect_beds_and_print(bed1, bed2, window=0, wa=True, wb=True):
    # construct interval trees for each chromosome
    data = np.loadtxt(bed2, dtype=str)
    chroms = data[:, 0].astype(str)
    starts = data[:, 1].astype(int)
    stops = data[:, 2].astype(int)

    chr2interval_tree = collections.defaultdict(IntervalTree)
    for i in range(len(data)):
        chr2interval_tree[chroms[i]].insert_interval(Interval(starts[i], stops[i], chrom=chroms[i]))

    # get overlapping intervals
    data2 = np.loadtxt(bed1, dtype=str)
    chroms2 = data2[:, 0].astype(str)
    starts2 = data2[:, 1].astype(int)
    stops2 = data2[:, 2].astype(int)

    for i in range(len(data2)):
        intervals = chr2interval_tree[chroms2[i]].find(starts2[i]-window, stops2[i]+window)
        if len(intervals) > 0:
            if wa and not wb:
                print "\t".join([chroms2[i], str(starts2[i]), str(stops2[i])])
            if wa and wb:
                for interval in intervals:
                    print "\t".join([chroms2[i], str(starts2[i]), str(stops2[i]),
                                     interval.chrom, str(interval.start), str(interval.end)])

First test: intersect of a.bed and b.bed


In [68]:
!bedtools intersect -a "a.bed" -b "b.bed" -wa -wb

chr1	100	200	a2	2	-	chr1	90	101	b2	2	-
chr1	100	200	a2	2	-	chr1	100	110	b3	3	+


In [69]:
intersect_beds_and_print("a.bed", "b.bed")

chr1	100	200	chr1	90	101
chr1	100	200	chr1	100	110


the results match.

Second test: intersect with window of 1

In [70]:
!bedtools window -a "a.bed" -b "b.bed" -w 1

chr1	10	20	a1	1	+	chr1	20	30	b1	1	+
chr1	100	200	a2	2	-	chr1	90	101	b2	2	-
chr1	100	200	a2	2	-	chr1	100	110	b3	3	+
chr1	100	200	a2	2	-	chr1	200	210	b4	4	+


In [71]:
intersect_beds_and_print("a.bed", "b.bed", window=1)

chr1	10	20	chr1	20	30
chr1	100	200	chr1	90	101
chr1	100	200	chr1	100	110
chr1	100	200	chr1	200	210


the results match.